In [1]:
%load_ext dotenv
%dotenv

In [3]:
from langchain_core.prompts import PromptTemplate
from langchain_core.output_parsers import StrOutputParser
from langchain_core.runnables import RunnablePassthrough, RunnableLambda
from langchain_core.runnables import chain

from langchain_openai import ChatOpenAI

from langchain.memory import ConversationSummaryMemory

from operator import itemgetter

In [5]:
TEMPLATE = '''
The following is a friendly conversation between a human and an AI. 
The AI is talkative and provides lots of specific details from its context. 
If the AI does not know the answer to a question, it truthfully says it does not know.

Current conversation:
{message_log}

Human: 
{question}

AI:
'''

prompt_template = PromptTemplate.from_template(template=TEMPLATE)

In [7]:
chat = ChatOpenAI(model_name = 'gpt-4o-mini', 
                  seed = 365,
                  temperature = 0,
                  max_tokens = 100)

In [9]:
chat_memory = ConversationSummaryMemory(llm = ChatOpenAI(),
                                        memory_key="message_log")

/var/folders/fs/ly3y_3m57537dkrh09vsr_mw0000gn/T/ipykernel_87784/3002585002.py:1: LangChainDeprecationWarning: Please see the migration guide at: https://python.langchain.com/docs/versions/migrating_memory/
  chat_memory = ConversationSummaryMemory(llm = ChatOpenAI(),


In [41]:
# question = "Can you give me an interesting fact I probably didn't know about?"
question = "Can you elaborate a bit more on this fact?"

In [43]:
dictionary_output = RunnablePassthrough.assign(message_log=RunnableLambda(chat_memory.load_memory_variables) |
                          itemgetter('message_log')).invoke({'question':question})

In [45]:
prompt_value_output = prompt_template.invoke(dictionary_output)

In [47]:
ai_message_output=chat.invoke(prompt_value_output)

In [48]:
response = StrOutputParser().invoke(ai_message_output)

In [49]:
chat_memory.save_context(inputs = {'inputs':question},
                         outputs = {'outputs':response}
                        )

In [50]:
chat_memory.load_memory_variables({})

{'message_log': "The human asks the AI for an interesting fact they probably didn't know about, and the AI shares that honey never spoils because of its low moisture content and acidic pH. The human then asks the AI to elaborate, and the AI explains that honey's longevity is due to its unique chemical composition, low moisture content, and acidic pH levels. Bees create honey by collecting nectar from flowers and enzymatically transforming it into the sweet substance we know."}

In [61]:
chain1 = (
    RunnablePassthrough.assign(
        message_log = RunnableLambda(chat_memory.load_memory_variables) | 
        itemgetter('message_log')) 
    | prompt_template 
    | chat 
    | StrOutputParser()
)

# question = "Can you give me an interesting fact I probably didn't know about?"
question = "Can you elaborate a bit more on this fact?"

response = chain1.invoke({'question':question})

chat_memory.save_context(inputs = {'input':question}, 
                         outputs = {'output':response})
response

"Absolutely! Honey's remarkable longevity is primarily due to its unique chemical composition. It has a low moisture content, typically around 17-18%, which is not enough to support the growth of most bacteria and fungi. Additionally, honey is naturally acidic, with a pH level ranging from about 3.2 to 4.5. This acidity further inhibits microbial growth.\n\nAnother fascinating aspect of honey is the process by which bees create it. Bees collect nectar from flowers and then enzymatically convert it"

In [15]:
@chain
def memory_chain(question):
    
    chain1 = (
        RunnablePassthrough.assign(
            message_log = RunnableLambda(chat_memory.load_memory_variables) | 
            itemgetter('message_log')) 
        | prompt_template 
        | chat 
        | StrOutputParser()
    )
    
    response = chain1.invoke({'question':question})

    chat_memory.save_context(inputs = {'input':question}, 
                             outputs = {'output':response})

    return response

In [19]:
memory_chain.invoke("Can you elaborate a bit more on this fact?")

"Absolutely! Honey's remarkable longevity is primarily due to its unique chemical composition. The low moisture content in honey, typically around 17-18%, creates an environment that is inhospitable for bacteria and other microorganisms. Additionally, honey has a natural acidity, with a pH level ranging from about 3.2 to 4.5, which further prevents the growth of harmful organisms.\n\nAnother fascinating aspect is that honey contains hydrogen peroxide, which is produced when the enzyme glucose oxidase breaks down glucose"